# Codificador Automático

In [3]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [97]:
def folder_to_sequence(folder_path: str, window_size: int) -> np.ndarray:
    """
    Função usada para converter os dados .csv de uma pasta
    para o formato aceito pelo autocodificador LSTM.

    Parameters
    ----------
    folder_path: str
        Caminho (pasta) em que os dados estão localizados.

    window_size: int
        Tamanho da janela de dados que o modelo receberá.

    Returns
    -------
    np.ndarray
        Array contendo as sequencias na forma de sequencias
    """

    X = None
    
    for filename in os.listdir("data/" + folder_path):
        if filename.endswith(".csv"):
            df_old = pd.read_csv("data/"+ folder_path + "/" + filename)
            df = calibrator.apply_calibration(df_old)
            sequences = df_to_sequence(df.PT105, window_size)
            if X is None:
                X = sequences
            else:
                X = np.concatenate((X, sequences))
    return X

def df_to_sequence(data: pd.DataFrame, window_size: int) -> np.ndarray:
    x = []
    for i in range(len(data) - window_size):
        row = [[r] for r in data[i:i+window_size]]  
        x.append(row)
        
    return np.array(x)

In [52]:
class Calibrator:
    
    def __init__(self):
        files = ["PT105"]
        self.data: dict = {}
        for filename in files:
            df = pd.read_csv(f"data/calibracao/{filename}.csv", delimiter=",", decimal=",")
            x, y = df.iloc[:, 0], df.iloc[:, 1]
            self.a, self.b = np.polyfit(x, y, 1)
            self.data[filename] = (self.a, self.b)
            
    def __str__(self) -> str:
        return str(self.data)

    def apply_calibration(self, df: pd.DataFrame) -> pd.DataFrame:
        df["PT105"] = self.a * df["cDAQ1Mod1/ai2"] + self.b
        return df

In [53]:
# ler a pasta
calibrator = Calibrator()

In [100]:
bigX = folder_to_sequence("VIDRO-B2", 100)
print("model shape: ", bigX.shape)

model shape:  (9070, 100, 1)
